# pySEAFOM Fidelity Test — THD (Synthetic Data)

This notebook demonstrates a **fidelity / THD** test workflow using the `pySEAFOM.fidelity` module.

## What this notebook does
- Generates synthetic DAS-like time series with a known stimulus frequency and controllable harmonic distortion
- Defines **test sections** (channel ranges) and **levels** (time windows)
- Slices out one section at a time and computes THD via `calculate_fidelity_thd()`
- Prints a compact report via `report_fidelity_thd()`

## 1) Setup and Imports

Adds the local `source` directory to `sys.path` so you can run this notebook without installing the package.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent
src_dir = repo_root / 'source'/ 'pySEAFOM'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

import importlib
import fidelity  # if using relative import
importlib.reload(fidelity)

#from pySEAFOM import fidelity # if using installed package import

## 2) Configure Test Parameters

This mirrors the structure you described: sections + channel ranges + level time windows + stimulus frequency.

In [ ]:
# Sampling
fs = 10000

# Sections
test_sections = ['Section 1', 'Section 2']
strecher_sections_channels=[[25,35],[65,75]]

# Levels (sample indices)
levels_time_steps = [
    [0, fs * 60],
    [int(1.1 * fs * 60), int(2.5 * fs * 60)],
]

# Stimulus + quality gate
stimulus_freq = 500
threshold = -40  # SNR threshold in dB for accepting FFT blocks

## 3) Create Synthetic Data

We synthesize a sinusoid at `stimulus_freq` and add harmonics.
To make the fidelity test meaningful, we create **two levels** with different harmonic ratios (low THD vs high THD).

In [ ]:
# ================================
# Harmonic Fidelity Test – Full Dataset Generator
# ================================

import numpy as np
import matplotlib.pyplot as plt

# ----------------
# Global parameters
# ----------------
n_channels = 100
duration_s = 160
N = int(fs * duration_s)

rng = np.random.default_rng(123)
t = np.arange(N) / fs

# ----------------
# Harmonic ratios
# ----------------
ratios_level0 = np.array([0.01, 0.005, 0.003, 0.002], dtype=float)
ratios_level1 = np.array([0.05, 0.02, 0.01, 0.005], dtype=float)

A1 = 1.0   # fundamental amplitude
ratios_list = [ratios_level0, ratios_level1]
# ----------------
# Section generator
# ----------------
def make_harmonic_section(start_ch, end_ch, levels_time_steps, ratios_list,
                          noise_sigma=0.05):
    """
    Build a multi-channel section with harmonic content that changes by time level.
    """
    n_ssl = end_ch - start_ch + 1
    section = np.zeros((n_ssl, N), dtype=np.float32)

    # Build 1D reference signal
    sig = np.zeros(N, dtype=np.float32)

    for (t0, t1), ratios in zip(levels_time_steps, ratios_list):
        tt = t[t0:t1]
        base = (A1 * np.sin(2 * np.pi * stimulus_freq * tt)).astype(np.float32)

        # Add harmonics
        for k, r in enumerate(ratios, start=2):
            phase = float(2 * np.pi * rng.random())
            base += (A1 * r * np.sin(2 * np.pi * k * stimulus_freq * tt + phase)).astype(np.float32)

        sig[t0:t1] += base

    # Add broadband noise
    sig += (noise_sigma * rng.standard_normal(N)).astype(np.float32)

    # Expand to multi-channel with gain variation
    gains = (1.0 + 0.02 * rng.standard_normal(n_ssl)).astype(np.float32)
    for i in range(n_ssl):
        section[i, :] = gains[i] * sig + (0.01 * rng.standard_normal(N)).astype(np.float32)

    return section
# ----------------
# Build all sections
# ----------------
sections = []
for (start_ch, end_ch) in strecher_sections_channels:
    sec = make_harmonic_section(start_ch, end_ch,
                                levels_time_steps,
                                ratios_list)
    sections.append(sec)

# ----------------
# Allocate full array + background noise
# ----------------
background_sigma = 0.2
data = background_sigma * rng.standard_normal((n_channels, N)).astype(np.float32)

# ----------------
# Insert sections into full array
# ----------------
for (start_ch, end_ch), sec in zip(strecher_sections_channels, sections):
    row_start = start_ch
    row_end   = row_start + sec.shape[0]
    data[row_start:row_end, :] = sec

# ----------------
# Visualization
# ----------------
plt.figure(figsize=(12, 4))
plt.imshow(data[:, :4000], aspect='auto', cmap='seismic')
plt.colorbar()
plt.title("Harmonic Fidelity Test – Full Data Array")
plt.xlabel("Samples")
plt.ylabel("Channel index")
plt.show()


## 4) Run Fidelity (THD) Computation

For each spatial section, we slice out the channel range and call `calculate_fidelity_thd()` on that single section.

`levels_time_steps` can be either:
- one window `[t0, t1)` (compute THD for one level), or
- a list of windows `[[t0, t1), ...]` (compute THD across multiple levels).

In [ ]:
# Per-section call: slice the channels yourself, and pass only that section
for i in range(len(test_sections)):
    ch0, ch1 = strecher_sections_channels[i]
    test_section = data[ch0:ch1 + 1, :]

    results = fidelity.calculate_fidelity_thd(
        test_section,
        fs=fs,
        levels_time_steps=levels_time_steps,  # list of windows -> multiple THD levels
        stimulus_freq=stimulus_freq,
        snr_threshold_db=-100,
        section_name=test_sections[i],
    )

    fidelity.report_fidelity_thd(results, show_harmonics=False)